In [1]:
## classic pydata stack
import os 
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys
%matplotlib inline 

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15,7)



## torch
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## SEEDING

torch.manual_seed(1)


REBUILD_DATA = True

In [2]:
def count_extremums(row):
    current=row[:,1]
    if len(current)<3:
        return 0
    increasing = current[0]<current[1]
    tmp=current[0]
    counter = 0
    values=[]
    for i in current[1:]:
        if increasing and i<tmp:
            increasing = False
            counter +=1
            values.append(i)
        elif not increasing and i>tmp:
            increasing=True
            counter+=1
            values.append(i)

        tmp=i
    return counter

def max_slope(row):
    maxslope=-1000
    for i in range(len(row)-1):
        if (row[i+1,0]-row[i,0])!=0 and (row[i+1,1]-row[i,1])/(row[i+1,0]-row[i,0])>maxslope:
            maxslope=(row[i+1,1]-row[i,1])/(row[i+1,0]-row[i,0])
    return maxslope
    
def min_slope(row):
    minslope=1000
    for i in range(len(row)-1):
        if (row[i+1,0]-row[i,0])!=0 and (row[i+1,1]-row[i,1])/(row[i+1,0]-row[i,0])<minslope:
            minslope=(row[i+1,1]-row[i,1])/(row[i+1,0]-row[i,0])

    return minslope
    

In [3]:
class Input():
    def __init__(self, raw_series,num_blocks,label):
        """ Initilaizes an input object from a raw time series i.e. an input suitable to feed to a recurrent neural network

        Args:
            raw_series (numpy array of shape (num_timesteps,2)): raw time series from npy data i.e. arr[0] where arr = np.load("data.npy")
            num_blocks ([type]): number of "feature blocks" into which the time series will be sliced i.e the number of of times we need to feed 
            to the LSTM to train on the entire time series
            label ([type]): Whether it was a "00" backbone (label:0) or a "66" backbone (label:1)
        """

        self.label = label
        self.input = self.process(raw_series,num_blocks)


    def process(self,raw_series,num_blocks):
        """ Function that does the entire processing of going from raw time series to a suitable input to feed to a recurrent neural network

        Args:
            raw_series (numpy array of shape (num_timesteps,2)): raw time series from npy data i.e. arr[0] where arr = np.load("data.npy")
            num_blocks ([type]): number of "feature blocks" into which the time series will be sliced i.e the number of of times we need to feed 
            to the LSTM to train on the entire time series

        Returns:
            np.ndarray: array of features from a single raw time series instance
        """


        # stores the processed time series
        res = np.array([])

        ## returns a list of transformed time series (current list: normal. lowpass filtered, highpass filtered)
        instances = self.transform(raw_series)


        for instance in instances:
            ## chunks an instance of a time series into blocks and extract feature from each block
            extracted = self.extract_features(instance,num_blocks)
            res = np.concatenate((res,extracted),axis=None)

        return res


    def transform(self,raw_series):
        """ Given a raw time series, outputs several transformations applied to it
            Transformations may be filtering, projecting, ...

        Args:
            raw_series numpy.ndarray : 1 dimensional array representing the current values

        Returns:
            List(numpy.ndarray): list of all transformations
        """

        res = [raw_series]

        return res

    def extract_features(self,instance, num_blocks):

        """ From a time series, divides it into num_blocks blocks and from each block, extract numerical features usable for a neural network

        Args:
            instance (numpy.ndarray): 1D array containing numerical values
            num_blocks (int): number of "feature blocks" into which the time series will be sliced i.e the number of of times we need to feed 
            to the LSTM to train on the entire time series


        Returns:
            numpy.ndarray: 1D array of length num_blocks*num_features_per_block containing all the features from a time series
        """

        res = np.array([])
        length = len(instance)
        # divide the length by num_blocks to get block_size
        block_size, remainder  = divmod(length,num_blocks)


        # iterating over each block and extracting features
        for i in range(num_blocks):

            curr = instance[block_size*i: block_size*(i+1)]
            # get features from block (mean, std, length, ...)
            features = self.features(curr)
            res = np.concatenate((res, features),axis=None)


        ## get the remainder of the time series
        ##curr = instance[block_size*num_blocks:]
        ##features = self.features(curr)
        ##res = np.concatenate((res,features),axis=None)

        return res

    def features(self,instance):
        """
        From a block of a time series, extracts numerical features usable for a neural network
        Args:
            instance (numpy.ndarray): 1D array containing numerical values 
        """
        res = np.array([])

        # list of functions applied to the array for feature extraction
        functions = [np.mean,np.median,np.std,np.min,np.max,len,count_extremums,max_slope, min_slope]

        for func in functions:
            res = np.concatenate((res,func(instance)),axis=None)

        

        return res        

In [4]:
from torch.utils.data import Dataset

class PolymerDataset(Dataset):

    ## These functions are necessary to define an iterator usable by Pytorch

    def __init__(self, data_paths,num_blocks, lstm=False, seed=10):
        super().__init__()
        self.process(data_paths,num_blocks,lstm,seed)

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


    def process(self,data_paths, num_blocks,lstm,seed):
        """ Processes the two datasets in the aim of not having bias catchable by the neural network:
        - filtering signals that are too long and too short
        - balancing the two datasets, resulting in the two classes each representing 50% of the data
        - Process each of the raw time series current into suitable inputs
        - Output a dataset where each row represents a suitable input for a NN derived from the raw time series

        Args:
            data_paths (list[string]): Should be a list of length 2 containing the paths of the data to be loaded 
            num_blocks (int): number of "feature blocks" into which a raw time series will be sliced i.e the number of of times we need to feed the
            to the LSTM to train on the entire time series
            seed (int): for setting the seed

        """
        
        raw_data = [np.load(data_path, allow_pickle=True) for data_path in data_paths]
        labels = [0,1]

    ## balance the dataset by removing signals that are too short or too long
    ## first we build the dataframe to know the lengths of the time series

        len_series = []

        for data in raw_data:
            lengths= []
            for row in data:
                ## length of time series
                lengths.append(row.shape[0])

            len_series.append(pd.Series(lengths))

        ## enforces that the first dataset is the smaller one in total size
        ## such that we can apply our balancing operations generally
        if len(len_series[0]) > len(len_series[1]):
            len_series.reverse()
            raw_data.reverse()
            labels.reverse()

        ## filter the dataset and remove signals that are:
        ## too short i.e. < len_series[0].quantile(0.1)
        ## too long i.e. > len_series[0].quantile(0.9)
        for i in range(2):
            mask = (len_series[i] > max(len_series[0].quantile(0.1),num_blocks)) & (len_series[i] < len_series[0].quantile(0.9))
            raw_data[i] = raw_data[i][mask]

        ## most likely, one dataset is still bigger than the other one
        ## therefore, we randomly sample data from the bigger dataset to create a new dataset of the same size as the small one 
        np.random.seed(seed=seed)

        # making sure the smallest dataset is the first one
        if len(raw_data[0]) > len(raw_data[1]):
            raw_data.reverse()
            labels.reverse()

        # randomly sampling and making a balanced dataset
        raw_data[1]  = np.random.permutation(raw_data[1])[:len(raw_data[0])]
        data=[]
        data_labels=[]
        
        ## using our Input class to build the entire dataset and extracting features from each row
        for index, raw_data in enumerate(raw_data):
            for raw_series in raw_data:
                processed_series = Input(raw_series=raw_series,num_blocks=num_blocks,label=labels[index])
                data.append(processed_series.input)
                data_labels.append(labels[index])
        data = np.array(data)

        #normalizing features
        data = (data - data.mean(axis=0)) / data.std(axis=0)


        data = torch.Tensor(data).float()

        ## if lstm is true, set up the data such that it can easily be fed into a lstm
        if lstm:
            data = data.view((data.shape[0],num_blocks,-1))

        self.data = data
        self.labels = torch.Tensor(np.array(data_labels)).long()

        return self


    

In [32]:
from torch.utils.data import DataLoader

class LSTM(nn.Module):

    def __init__(self, input_dim, num_layers, hidden_dim):
        """ creates a lstm neural network

        Args:
            input_dim (int)): Defines the dimension of the input x, should be equal to the number of features extracted per block
            num_layers(int): Defines the number of LSTM layers, should be equal to num_blocks
            hidden_dim (int): defines the number of features in the hidden states
        """
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size= input_dim, num_layers=num_layers, hidden_size=hidden_dim,batch_first=True)
        self.fc1 =nn.Linear(hidden_dim, 2)

    
    def forward(self, input):
        """ Forward pass of our network

        Args:
            input ([type]): should be our current time series preprocessed with shape(num_blocks, num_features) 
            where num_blocks is the number of blocks in which we have divided our time series and  num_features is the number of feature per block
        """
        num_blocks=input.shape[0]
        
        ## the LSTM output are the hidden states values for all hidden states while processing the sequence
        lstm_out, _ = self.lstm(input)

        ## we only want last hidden states values
        lstm_out = lstm_out[:,-1,:]
        ## passing through MLP and softmax
        last = self.fc1(lstm_out.view(num_blocks,-1))
        scores = F.log_softmax(last,dim=1)

        return scores

    def predict(self, test_data):
        probs = self.forward(X)
        preds = torch.argmax(probs, dim=1, keepdim=False)
        return preds

    def train(dataset, num_features, num_blocks, hidden_dim, num_epochs, batch_size, lr=0.1, verbose="v"):

        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        model = LSTM(input_dim = num_features, num_layers= num_blocks ,hidden_dim = hidden_dim)
        loss_function = torch.nn.NLLLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)

        for epoch in range(num_epochs):
            num_correct = 0
            for X, y in iter(data_loader):
                model.zero_grad()
                probs = model(X)
                loss = loss_function(probs, y)
                loss.backward()
                optimizer.step()
                preds = torch.argmax(probs, dim=1, keepdim=False)
                num_correct += (preds == y).sum()
            if "vv" in verbose or ("v" in verbose and epoch%50==0) or epoch==num_epochs :
                print(f'epoch={epoch}/{num_epochs - 1}, loss={loss}, accuracy={num_correct*100/len(dataset)}')


        return model

In [6]:
num_blocks=5


dataset = PolymerDataset(data_paths=["../data/AA66266AA.npy","../data/AA662266AA.npy"],num_blocks=num_blocks,lstm=True)
num_features = dataset.data[0].shape[1]

from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])


In [7]:
model = LSTM.train(dataset=train_data, num_features=num_features, num_blocks=num_blocks, hidden_dim=4, num_epochs=300, batch_size=64, lr=0.05)

epoch=0/299, loss=0.6944653987884521, accuracy=50.10301971435547
epoch=1/299, loss=0.6926047801971436, accuracy=49.87921905517578
epoch=2/299, loss=0.6923667192459106, accuracy=50.394317626953125
epoch=3/299, loss=0.689285159111023, accuracy=50.54351806640625
epoch=4/299, loss=0.6925758123397827, accuracy=50.51865005493164
epoch=5/299, loss=0.6940355896949768, accuracy=50.309059143066406
epoch=6/299, loss=0.6934224367141724, accuracy=50.57548904418945
epoch=7/299, loss=0.6925137639045715, accuracy=50.98756790161133
epoch=8/299, loss=0.6929244995117188, accuracy=50.9094123840332
epoch=9/299, loss=0.6925257444381714, accuracy=50.991119384765625
epoch=10/299, loss=0.6923526525497437, accuracy=51.4422721862793
epoch=11/299, loss=0.6904265284538269, accuracy=52.92007064819336
epoch=12/299, loss=0.6979155540466309, accuracy=55.06571960449219
epoch=13/299, loss=0.6798733472824097, accuracy=61.49555969238281
epoch=14/299, loss=0.36901745200157166, accuracy=75.54174041748047
epoch=15/299, loss=

In [8]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

0.9485649332196647 0.9489421720733426
[[3312  171]
 [ 191 3364]]


In [43]:
def test_model(model, data_loader):
    predictions = np.array([])
    labels = np.array([])

    with torch.no_grad():
        for X, y in iter(data_loader):
            probs = model(X)
            preds = torch.argmax(probs, dim=1, keepdim=False)
            predictions = np.concatenate((predictions,preds), axis=None)
            labels= np.concatenate((labels,y),axis=None)

    return(accuracy_score(labels,predictions), f1_score(labels,predictions))

    print(confusion_matrix(labels,predictions))


In [44]:
import itertools as it

In [45]:
params_list={"num_blocks":[1,2,3,4,5,6],
"lr":[0.1,0.05,0.01],
"hidden_dim":[1,2,3,4,5],
"num_epochs":[200]
}

In [46]:
def grid_search(params_list):
    param_combinations = list(it.product(*(params_list[param_name] for param_name in params_list.keys())))

    best_accuracy=0
    last_num_blocks=-1

    for params in param_combinations:
        kwargs={}
        for i,key in enumerate(params_list):
            kwargs[key]=params[i]
        if kwargs.get("num_blocks",1)!=last_num_blocks:
            dataset = PolymerDataset(data_paths=["../data/AA66266AA.npy","../data/AA662266AA.npy"],num_blocks=kwargs.get("num_blocks",1),lstm=True)
            train_size = int(0.8 * len(dataset))
            test_size = len(dataset) - train_size
            train_data, test_data = random_split(dataset, [train_size, test_size])
            data_loader = DataLoader(test_data, batch_size=64, shuffle=False)

             
        model = LSTM.train(dataset=train_data, num_features=num_features, batch_size=64, **kwargs)
        test_accuracy, test_f1 = test_model(model, data_loader)
        if test_accuracy>best_accuracy:
            best_accuracy = test_accuracy
            best_f1=test_f1
            best_model=model
            best_params=kwargs
        last_num_blocks=kwargs.get("num_blocks",1)
    print("Best accuracy ({}) and f1 ({}) were reached with params {}".format(best_accuracy,best_f1, best_params))
    return best_model, best_params


In [47]:
grid_search(params_list)

epoch=0/199, loss=0.4627956748008728, accuracy=67.85079956054688
epoch=50/199, loss=0.24814613163471222, accuracy=89.26820373535156
epoch=100/199, loss=0.16452254354953766, accuracy=89.32149505615234
epoch=150/199, loss=0.3650842010974884, accuracy=89.35701751708984
epoch=0/199, loss=0.3733663558959961, accuracy=76.316162109375
epoch=50/199, loss=0.11893614381551743, accuracy=92.21314239501953
epoch=100/199, loss=0.2790621221065521, accuracy=92.38721466064453
